In [1]:
# import os, getpass
from ibm_watsonx_ai import Credentials, APIClient
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_ibm import ChatWatsonx
# from langchain.chains import RetrievalQAjin
from ibm_watsonx_ai.foundation_models.schema import TextChatParameters
from langchain_experimental.text_splitter import SemanticChunker
from langchain_huggingface import HuggingFaceEmbeddings

from docling.document_converter import DocumentConverter


/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load documents

In [2]:
file_path = "dummy2.pdf"
converter = DocumentConverter()
result = converter.convert(file_path)
# doc = result.document


/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:262: RuntimeWarning: Mean of empty slice
  np.nanmean(
/home/behrad/miniforge3/envs/ibm/lib/python3.12/site-packages/docling/pipeline/standard_pdf_pipeline.py:267: RuntimeWarning: Mean of empty slice
  np.nanmean(


## Chunk

In [ ]:
embedder = HuggingFaceEmbeddings(model="jinaai/jina-embeddings-v3", model_kwargs={"trust_remote_code": True})

In [23]:
import numpy as np
texts = [
    "Follow the white rabbit.",  # English
    "Sigue al conejo blanco.",  # Spanish
    "Suis le lapin blanc.",  # French
    "跟着白兔走。",  # Chinese
    "اتبع الأرنب الأبيض.",  # Arabic
    "Folge dem weißen Kaninchen.",  # German
    "خرگوش سفید را دنبال کن"

]

embeddings = embedder.embed_documents(texts)
embeddings = np.array(embeddings)
print(embeddings[0] @ embeddings[1].T)

0.7086319464168012


In [24]:
print(embeddings[0] @ embeddings[6].T)

0.64758606593628


In [20]:
import numpy as np
texts = [
    "Follow the white rabbit.",  # English
    "Sigue al conejo blanco.",  # Spanish
    "Suis le lapin blanc.",  # French
    "跟着白兔走。",  # Chinese
    "اتبع الأرنب الأبيض.",  # Arabic
    "Folge dem weißen Kaninchen.",  # German
    "خرگوش سفید را دنبال کن"

]

embeddings = embedder.embed_documents(texts)
embeddings = np.array(embeddings)
print(embeddings[0] @ embeddings[1].T)

0.6718303814857509


In [21]:
print(embeddings[0] @ embeddings[6].T)

0.5915971370015054


In [6]:
text_splitter = SemanticChunker(embedder)

In [ ]:
api_key="VPryyL4t508YQ_pJQNCU0m23Yctctahrqo2pjT5BMQkV"
url="https://us-south.ml.cloud.ibm.com"

# credentials = Credentials(
#     url=url,
#     api_key=api_key
# )
project_id = "6552c3bb-a35a-4bf3-8430-c42828d59a06"
# api_client = APIClient(credentials=credentials, project_id=project_id)

In [ ]:
# filename = "data/email.csv"
# loader = TextLoader(filename)
# documents = loader.load()
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# texts = text_splitter.split_documents(documents)
# texts

In [ ]:
model_id = "meta-llama/llama-4-maverick-17b-128e-instruct-fp8"
parameters = TextChatParameters(
    max_tokens = 1024,
    temperature = 0
)
watsonx_llm = ChatWatsonx(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params=parameters,
)

In [ ]:
messages = [
    ("system", "You are a helpful assistant that translates English to French."),
    (
        "human",
        "I love you for listening to Rock.",
    ),
]


In [ ]:

out = watsonx_llm.invoke(messages)


In [ ]:
out.content